In [1]:
!pip cache purge

Files removed: 18


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The Library Versions

1. Numpy = 2.3.4 --> 1.26.4
2. Torch = 2.9.0 --> 2.8.0+cu126
3. Transforers = 4.57.1 --> 4.51.2
4. peft = 0.17.1 -->
5. bitsandbytes --> 0.48.1

In [3]:
!pip uninstall -y numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [4]:
!pip install --no-cache-dir -q numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 115.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requi

In [5]:
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


In [6]:
!pip install -q torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu126

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.8/821.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 72.8 MB/s eta 0:00:00


In [7]:
!pip uninstall -y transformers

Found existing installation: transformers 4.51.2
Uninstalling transformers-4.51.2:
  Successfully uninstalled transformers-4.51.2


In [8]:
!pip install -q transformers==4.51.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 86.2 MB/s eta 0:00:00


In [9]:
!pip install -q bitsandbytes==0.48.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.0 MB/s eta 0:00:00


In [10]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.48.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, packaging, torch
Required-by: 


In [11]:
import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from peft import PeftModel

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
OUTPUT_DIR = "/content/drive/MyDrive/WEBBIES_NOTEBOOKS/PERSONAL_RESEARCH/Assessli_Research/New_LBM/Checkpoints"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token

In [15]:
# Load base model (same as during training)
model_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "cuda",
    trust_remote_code = True,
    load_in_4bit = True,  # if using 4-bit quantization
    bnb_4bit_compute_dtype = torch.bfloat16,
    low_cpu_mem_usage = True
)

# Load LoRA adapter on top of base model
model = PeftModel.from_pretrained(model_base, OUTPUT_DIR)
model.eval()
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

print("The model is loaded in Evaluation Mode with LoRA adapter.")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

The model is loaded in Evaluation Mode with LoRA adapter.


In [16]:
# Define a function to generate summary
def generate_summary(document: str, user_profile: dict, max_new_tokens = 200):
    profile_str = f"<FOCUS:{user_profile['focus']}> <TONE:{user_profile['tone']}> <LENGTH:{user_profile['length']}> <HISTORY:{'|'.join(user_profile['history'])}>"
    prompt = f"Summarize the following document for a user with preferences: {profile_str}\n\nDocument:\n{document}\n\nSummary:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    gen_config = GenerationConfig(
        temperature=0.2,
        top_p=0.95,
        do_sample=False,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id
    )
    with torch.no_grad():
        out = model.generate(**inputs, generation_config=gen_config)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # post-process: strip prompt prefix
    summary_text = text.split("Summary:")[-1].strip()
    return summary_text

In [17]:
sample_doc = """Nearly 2,000 miners across four states may lose their jobs after yet another major coal company filed for bankruptcy this week
the third since May and fourth since last October. The bankruptcy filing from Revelation Energy LLC and its affiliate Blackjewel LLC,
the nation's sixth-top coal producing company in 2017, comes amid President Donald Trump's ongoing efforts to boost the flagging industry.
The Trump administration rolled out a rule last month aiming to extend the lives of aging coal-fired power plants across the nation.
Environmentalists say the Affordable Clean Energy rule would trigger premature deaths, including from lung disease. During his 2016
presidential campaign, Trump promised to revitalize coal and save miners' jobs, despite scientists linking the burning of the fossil fuels
to global warming , but the industry has continued to suffer losses. Coal comeback? Trump plan breathes new life into aging power plants,
but critics say climate will suffer At mines and facilities in Virginia, Kentucky, West Virginia and Wyoming, Revelation Energy and
Blackjewel employ 1,800 workers, according to court documents and The Casper (Wyo.) Star-Tribune . Company officials estimate they owe
$156 million for goods and services, West Virginia Public Radio reported. Last month, Cambrian Coal LLC also filed for bankruptcy .
The company operating in Kentucky and Virginia blamed its bankruptcy on changes in demand and regulations related to the Clean Air Act.
Another coal-producing leader filed for Chapter 11 bankruptcy in May. Once the nation's third-largest coal company, Wyoming-based Cloud Peak Energy
employed 1,300 people at the time of its filing. It accounted for 7.4% of total U.S. coal production in 2017, according to the Department of Labor .
And, the nation's ninth-leading coal company went to bankruptcy court late in 2018. Colorado-based Westmoreland Coal Co. had more than $1.4
billion in debt at the time, The Associated Press reported. Gone by 2030?: On World Environment Day, everything you know about energy in
the US might be wrong Although Trump has touted coal's rebirth, 51 coal plants have closed and eight coal companies have filed for bankruptcy
since his election, CBS News reported last month. Coal's share of the U.S. electricity mix fell from 48% in 2008 to 27% in 2018 and is
projected to be 22% in 2020, according to the Department of Energy. \"We're retiring a coal plant every month. Coal will all be gone by 2030,
said Bruce Nilles , a managing director at the Rocky Mountain Institute, a think tank in Colorado that focuses on energy and resource efficiency.
Coal policy, including Trump's Affordable Clean Energy rule , could influence the 2020 election in swing states where coal is still mined,
such as Ohio and Pennsylvania. Contributing: Beth Weise and Ledyard King, USA TODAY This article originally appeared on USA TODAY:
Is President Donald Trump losing his fight to save coal? Third major company since May files for bankruptcy.
"""

sample_doc[:500]

"Nearly 2,000 miners across four states may lose their jobs after yet another major coal company filed for bankruptcy this week\nthe third since May and fourth since last October. The bankruptcy filing from Revelation Energy LLC and its affiliate Blackjewel LLC,\nthe nation's sixth-top coal producing company in 2017, comes amid President Donald Trump's ongoing efforts to boost the flagging industry.\nThe Trump administration rolled out a rule last month aiming to extend the lives of aging coal-fired"

In [18]:
# Build a demo user profile based on the sample_doc content
demo_profile = {
    "focus": "results",      # focus on the outcome of the events: job losses, bankruptcies
    "tone": "analytical",    # factual, professional summary style
    "length": "medium",      # medium length to capture key points
    "history": [
        "Coal industry layoffs in Appalachia",
        "Bankruptcy filings of major mining companies",
        "Trump administration energy policies",
        "Impact of Clean Air Act on coal sector",
        "US electricity mix and coal decline"
    ]
}

print("Demo user profile:\n", demo_profile)

Demo user profile:
 {'focus': 'results', 'tone': 'analytical', 'length': 'medium', 'history': ['Coal industry layoffs in Appalachia', 'Bankruptcy filings of major mining companies', 'Trump administration energy policies', 'Impact of Clean Air Act on coal sector', 'US electricity mix and coal decline']}


In [19]:
demo_profile = {"focus":"results","tone":"analytical","length":"short","history":["previous_article_title_1"]}
print("Generated summary:\n", generate_summary(sample_doc, demo_profile))

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configurati

Generated summary:
 The article discusses the recent bankruptcy filing of Revelation Energy LLC and Blackjewel LLC, two major coal companies in the United States. The companies employ 1,800 workers across four states and owe $156 million in goods and services. This is the third major coal company to file for bankruptcy since May, and the fourth since last October. The article highlights President Donald Trump's efforts to boost the flagging coal industry, including a recent rule aimed at extending the lives of aging coal-fired power plants. However, despite these efforts, the coal industry continues to suffer losses due to changing demand and regulations related to the Clean Air Act. The article also notes that coal's share of the US electricity mix is projected to be 22% in 2020, down from 48% in 2008. The article concludes that
